In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/scamps_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [15]:
phys_net = PhysNet()
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')
phys_net.summary()

Flops per frame:54260867
Model: "phys_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_22 (Sequential)   (None, 128, 32, 32, 16)   1280      
_________________________________________________________________
sequential_23 (Sequential)   (None, 128, 16, 16, 32)   13984     
_________________________________________________________________
sequential_24 (Sequential)   (None, 128, 16, 16, 64)   55616     
_________________________________________________________________
sequential_25 (Sequential)   (None, 64, 8, 8, 64)      110912    
_________________________________________________________________
sequential_26 (Sequential)   (None, 64, 8, 8, 64)      110912    
_________________________________________________________________
sequential_27 (Sequential)   (None, 32, 4, 4, 64)      110912    
_________________________________________________________________
sequential_28 (Sequential)   (N

In [16]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=1, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet_SCAMPS.h5')

374/374 [==============================] - 186s 477ms/step - loss: 0.8915 - val_loss: 0.5647

Epoch 00001: val_loss improved from inf to 0.56465, saving model to ../weights\PhysNet_SCAMPS.h5


In [7]:
phys_net.load_weights('../weights/PhysNet_SCAMPS.h5')

In [17]:
eval_on_dataset(test_set_PURE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_SCAMPS_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_SCAMPS_PURE.h5')

100%|██████████| 59/59 [00:24<00:00,  2.37it/s]


{'Sliding window': {'MAE': 37.317, 'RMSE': 41.773, 'R': -0.00354},
 'Whole video': {'MAE': 36.252, 'RMSE': 40.709, 'R': 0.13516}}

In [18]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_SCAMPS_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_SCAMPS_UBFC.h5')

100%|██████████| 42/42 [00:16<00:00,  2.49it/s]


{'Sliding window': {'MAE': 15.718, 'RMSE': 21.101, 'R': 0.17538},
 'Whole video': {'MAE': 13.733, 'RMSE': 18.966, 'R': 0.24}}